In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.min_rows', 100)

**Problem 1**

Based on Table in above Scenario, estimate the expected win rate for a bid repsonse at a given price (prices are listed in table). State your assumptions clearly and provide the steps on how you arrived at your answer.

**Problem 2**

We receive money from our advertisers if we deliver them a win. Lets say that our advertiser is willing to pay $0.50 per win. This then becomes the upper bound for the bid valuation that we can submit in response to the publisher. For example, if we submit a bid response of $0.40 and we win, then the advertiser pays us $0.50, we pay $0.40 to the publisher and we made a net revenue of $0.10 .

If our goal was to maximize net revenue, what is the most optimal bid valuation we should send in our response? Use your estimations from Problem1 and all other available information.

Here is the json for Table 1 if you want to play around with it.

In [3]:
table = [
{"app": "A", "bid_price": 0.01, "win": 0, "events": 100000}, 
{"app": "A", "bid_price": 0.01, "win": 1, "events": 0}, 
{"app": "A", "bid_price": 0.1, "win": 0, "events": 7000}, 
{"app": "A", "bid_price": 0.1, "win": 1, "events": 3000}, 
{"app": "A", "bid_price": 0.2, "win": 0, "events": 8000000}, 
{"app": "A", "bid_price": 0.2, "win": 1, "events": 2000000}, 
{"app": "A", "bid_price": 0.4, "win": 0, "events": 700000}, 
{"app": "A", "bid_price": 0.4, "win": 1, "events": 300000}, 
{"app": "A", "bid_price": 0.5, "win": 0, "events": 80000}, 
{"app": "A", "bid_price": 0.5, "win": 1, "events": 20000}, 
{"app": "A", "bid_price": 0.75, "win": 0, "events": 7000}, 
{"app": "A", "bid_price": 0.75, "win": 1, "events": 3000}, 
{"app": "A", "bid_price": 1, "win": 0, "events": 400}, 
{"app": "A", "bid_price": 1, "win": 1, "events": 600}, 
{"app": "A", "bid_price": 2, "win": 0, "events": 30}, 
{"app": "A", "bid_price": 2, "win": 1, "events": 70}, 
{"app": "A", "bid_price": 5, "win": 0, "events": 2}, 
{"app": "A", "bid_price": 5, "win": 1, "events": 8}, 
{"app": "A", "bid_price": 9, "win": 0, "events": 0}, 
{"app": "A", "bid_price": 9, "win": 1, "events": 1}
]

In [4]:
df = pd.DataFrame(table)

## Part 1

In [5]:
df['total_events'] = np.cumsum(df['events'])
total_events_per_bid = pd.DataFrame(df.groupby('bid_price')['events'].sum()).reset_index() \
    .rename(columns={'events': 'total_events_per_bid', 'bid_price': 'bp'})
total_events_per_bid

,bp,total_events_per_bid
0,0.01,100000
1,0.10,10000
2,0.20,10000000
3,0.40,1000000
4,0.50,100000
5,0.75,10000
6,1.00,1000
7,2.00,100
8,5.00,10
9,9.00,1


In [6]:
df['total_win_prob'] = df.apply(lambda row: round(row.events/row.total_events, 3) if row.win==1 else 0, axis=1)
df['total_win_rate'] = df.apply(lambda row: round(row.events/row.total_events, 3) if row.win==0 else row.total_win_prob, axis=1)
df['total_odds'] = df['total_win_rate'].div(df.groupby('bid_price')['total_win_rate'].shift(1))

df = df.merge(total_events_per_bid, left_on='bid_price', right_on='bp').drop(['bp'], axis=1)
df['win_prob'] = df.apply(lambda row: round(row.events/row.total_events_per_bid, 3) if row.win==1 else 0, axis=1)
df['win_rate'] = df.apply(lambda row: round(row.events/row.total_events_per_bid, 3) if row.win_prob==0 else row.win_prob, axis=1)
df['odds'] = df['win_rate'].div(df.groupby('bid_price')['win_rate'].shift(1))

df = df.fillna(0)
df = df.drop(['total_win_prob', 'win_prob'], axis=1)
df

,app,bid_price,win,events,total_events,total_win_rate,total_odds,total_events_per_bid,win_rate,odds
0,A,0.01,0,100000,100000,1.000,0.000000,100000,1.0,0.000000
1,A,0.01,1,0,100000,0.000,0.000000,100000,0.0,0.000000
2,A,0.10,0,7000,107000,0.065,0.000000,10000,0.7,0.000000
3,A,0.10,1,3000,110000,0.027,0.415385,10000,0.3,0.428571
4,A,0.20,0,8000000,8110000,0.986,0.000000,10000000,0.8,0.000000
5,A,0.20,1,2000000,10110000,0.198,0.200811,10000000,0.2,0.250000
6,A,0.40,0,700000,10810000,0.065,0.000000,1000000,0.7,0.000000
7,A,0.40,1,300000,11110000,0.027,0.415385,1000000,0.3,0.428571
8,A,0.50,0,80000,11190000,0.007,0.000000,100000,0.8,0.000000
9,A,0.50,1,20000,11210000,0.002,0.285714,100000,0.2,0.250000


### Observations

**1. Assumption - Bidding events are sequential for given bid prices**

1. If bidding events are sequential (i.e total events as cummultive sum of all previous events observed till the current bid price), and we observe losses given bid price for n events and then m wins for the same bid price, and then shift to next higher bid price. In this case, we should observe win rate w.r.t overall events that were recorded up until current bid price
2. In that case, bid prices $0.1 and $0.4 have the highest odds of 0.415, but in a bidding scenario higher bid has more chances of winning even with same win rate. Also, the number of wins corresponding to $0.4 is more than that for $0.1 from the given data which means we start losing auctions with $0.1 more early even with the same win rate. 
3. So, the optimal price to bid is **$0.4**, because higher bid price, higher chances of winning in an auction scenario as compared to $0.1

- **Answer** - Expected win rates for each bid price is given in column **total_win_rate** of the above dataframe


**2. Assumption - Bidding events are independent for given bid prices**

1. If events are independent w.r.t to each bid price, then bidding should be done with bid price of highest win rate (4) i.e $5, ignoring $9 because only 1 event

- **Answer** - Expected win rates for each bid price is given in column **win_rate** of the above dataframe

## Part 2

In [7]:
df['profit_per_bid'] = 0.5-df['bid_price']
df['total_profit'] = df.apply(lambda row: row.events * row.profit_per_bid if row.win==1 else 0, axis=1)
df

,app,bid_price,win,events,total_events,total_win_rate,total_odds,total_events_per_bid,win_rate,odds,profit_per_bid,total_profit
0,A,0.01,0,100000,100000,1.000,0.000000,100000,1.0,0.000000,0.49,0.0
1,A,0.01,1,0,100000,0.000,0.000000,100000,0.0,0.000000,0.49,0.0
2,A,0.10,0,7000,107000,0.065,0.000000,10000,0.7,0.000000,0.40,0.0
3,A,0.10,1,3000,110000,0.027,0.415385,10000,0.3,0.428571,0.40,1200.0
4,A,0.20,0,8000000,8110000,0.986,0.000000,10000000,0.8,0.000000,0.30,0.0
5,A,0.20,1,2000000,10110000,0.198,0.200811,10000000,0.2,0.250000,0.30,600000.0
6,A,0.40,0,700000,10810000,0.065,0.000000,1000000,0.7,0.000000,0.10,0.0
7,A,0.40,1,300000,11110000,0.027,0.415385,1000000,0.3,0.428571,0.10,30000.0
8,A,0.50,0,80000,11190000,0.007,0.000000,100000,0.8,0.000000,0.00,0.0
9,A,0.50,1,20000,11210000,0.002,0.285714,100000,0.2,0.250000,0.00,0.0


### Observations 

1. From the above stats, we can see that although $0.4 has higher chances of winning but the the number of wins and the total revenue generated is more with bid price of $0.2.
2. This is because the numner of wins for $0.2 are far more than that of $0.4, the publisher might be more interested in paying $0.2 than $0.4 in an ideal scenario based on total number of wins. 

**Answer - $0.2 should be the optimal bid price for maximizing revenue.**
    

**Assumption - Bids were random events related to bid prices in an auction**

1. Assuming this data represents random bid events, we can simulate an env and understand whether we'll reach the maximum revenue possible given in the data
2. We'll create a MAB problem with arms as bid prices [0.01, 0.1, 0.2, 0.4, 0.5], and let the model learn how to generate maximum revenue based on profit gained at each random bid

In [8]:
# data01 = np.concatenate((np.ones(0), np.zeros(100000)))
# df_001 = pd.DataFrame(data01, columns=['0.01'])
# df_001['0.01'] = df_001['0.01'].replace(1, (0.5-0.01))
# print(df_001['0.01'].value_counts())

# data1 = np.concatenate((np.ones(3000), np.zeros(7000)))
# df_01 = pd.DataFrame(data1, columns=['0.1'])
# df_01['0.1'] = df_01['0.1'].replace(1, (0.5-0.1))
# print(df_01['0.1'].value_counts())

# data2 = np.concatenate((np.ones(2000000), np.zeros(8000000)))
# df_02 = pd.DataFrame(data2, columns=['0.2'])
# df_02['0.2'] = df_02['0.2'].replace(1, (0.5-0.2))
# print(df_02['0.2'].value_counts())

# data4 = np.concatenate((np.ones(300000), np.zeros(700000)))
# df_04 = pd.DataFrame(data4, columns=['0.4'])
# df_04['0.4'] = df_04['0.4'].replace(1, (0.5-0.4))
# print(df_04['0.4'].value_counts())

# data5 = np.concatenate((np.ones(20000), np.zeros(80000)))
# df_05 = pd.DataFrame(data5, columns=['0.5'])
# df_05['0.5'] = df_05['0.5'].replace(1, (0.5-0.5))
# print(df_05['0.5'].value_counts())

In [9]:
# final_df = pd.concat([df_001, df_01, df_02, df_04, df_05], axis=0)
# final_df = final_df.fillna(0)
# shuffled_df = final_df.sample(frac=1).reset_index(drop=True)
# print(shuffled_df.shape)

In [10]:
# shuffled_df.to_csv('simul_data.csv', index=None)

In [3]:
shuffled_df = pd.read_csv('dataset/simul_data.csv', index_col=None)
shuffled_df.head()

,0.01,0.1,0.2,0.4,0.5
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.3,0.0,0.0
4,0.0,0.0,0.3,0.0,0.0


In [4]:
N = 11110000
total_bids, bids_selected = 4, []
no_of_selections, sum_profit_bid = [0]*total_bids, [0]*total_bids
total_profit = 0

In [5]:
for n in range(0, N):
    bid, max_upper_bound = 0, 0
    
    for i in range(0, total_bids):
        if (no_of_selections[i] > 0):
            average_reward = sum_profit_bid[i] / no_of_selections[i]
            delta_i = math.sqrt(3/2 * math.log(n+1)  / no_of_selections[i])
            upper_bound = average_reward + delta_i
        else:
            upper_bound = 1e400
        
        if upper_bound > max_upper_bound:
            max_upper_bound = upper_bound
            bid = i

    bids_selected.append(bid)
    no_of_selections[bid] = no_of_selections[bid] +  1
    
    reward = shuffled_df.values[n, bid]
    sum_profit_bid[bid] = sum_profit_bid[bid] + reward
    total_profit = total_profit + reward
    
    if n%2000000==0:
        print(total_profit)
    

0.0
105898.80000065334
212626.99999773302
319627.2999935808
426482.29998943413
533704.9999871004


In [6]:
# plt.hist(bids_selected)
# plt.title('Bids selections')
# plt.xlabel('Bids')
# plt.ylabel('Number of times bid was selected')
# plt.show()

In [7]:
from collections import Counter
Counter(bids_selected)

Counter({0: 8045, 1: 8088, 2: 11084885, 3: 8982})